In [25]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .remote("sc://192.168.1.7:15002") \
  .appName("UDFTransformation") \
  .config("spark.sql.ansi.enabled", "false") \
  .getOrCreate()

# limit() shows a nice HTML table in Jupyter, while show() prints plain text  
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

spark

In [26]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
import pandas as pd
import pyspark.pandas as ps

def mult_by_3(x: int) -> int:
  return x * 3

# TODO THIS STEP IS NOT WITH PANDAS KEEP IN MIND
@pandas_udf(IntegerType())
def mult_by_3_udf(s: pd.Series) -> pd.Series:
  return s * 3

df = spark.createDataFrame([(4, ), (5, ), (6, )], ['value'])
df

value
4
5
6


In [27]:
df.withColumn('value_x3', mult_by_3_udf(df.value))

value,value_x3
4,12
5,15
6,18


In [4]:
import pandas